In [1]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/네이버 영화 리뷰

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/네이버 영화 리뷰


In [8]:
import pandas as pd
import time

import requests

from bs4 import BeautifulSoup
import re

# 웹페이지 요청 보내기
webpage = requests.get('https://movie.naver.com/movie/bi/mi/point.naver?code=74977')

# webpage.content 지정후 beatifulsoup 객체 저장
soup = BeautifulSoup(webpage.content, "html.parser")

# iframe scr 주소 확인
soup.find_all('iframe')

[<iframe class="ifr" frameborder="0" id="pointAfterListIframe" name="pointAfterListIframe" scrolling="no" src="/movie/bi/mi/pointWriteFormList.naver?code=74977&amp;type=after&amp;isActualPointWriteExecute=false&amp;isMileageSubscriptionAlready=false&amp;isMileageSubscriptionReject=false" title="네티즌 평점 리스트" width="100%"></iframe>]

In [9]:
# 앞뒤 공백 제거
def del_brank(sentence):
  while sentence.startswith('\r') | sentence.startswith('\t') | sentence.startswith('\n') == True:
    sentence = sentence[2:]

  while sentence.endswith('\r') | sentence.endswith('\t') | sentence.endswith('\n') == True:
    sentence = sentence[:-2]
    
  return sentence

# iframe의 src의 주소로 url 설정
def crawling(url):
  webpage = requests.get(url)
  soup = BeautifulSoup(webpage.content, "html.parser")

  # 총 영화 리뷰 수
  total_review = int(soup.select('body > div > div > div.score_total > strong > em')[0].text.replace(',',''))

  # 빈 데이터 프레임 생성
  movie_df = pd.DataFrame(columns=['score','sentence','id','date','heart','hate'])

  # 리뷰 크롤링
  start_time = time.time()
  for page in range(1, total_review//10 + 2):
    print('#########page######### : ', page)
    page_url = url + '&page=' + str(page)
    w_page = requests.get(page_url)
    soup = BeautifulSoup(w_page.content, 'html.parser')

    # 한 페이지에 존재하는 리뷰 개수
    review_count = 10
    if page == (total_review//10 + 1):
      review_count = total_review%10

    # 리뷰별 점수, 문장, 아이디, 날짜, 좋아요, 싫어요 수집
    for i in range(review_count):
      score = soup.select('body > div > div > div.score_result > ul > li > div.star_score > em')[i].text

      if len(soup.find_all('span',{'id' : '_unfold_ment' + str(i)})) > 0:
        sentence = soup.select('#_unfold_ment' + str(i)  + ' > a')[0].get('data-src')
      else:
        sentence = del_brank(soup.select('#_filtered_ment_' + str(i))[0].text)

      id = soup.select('body > div > div > div.score_result > ul > li > div.score_reple > dl > dt > em > a')[i].span.text
      date = soup.select('body > div > div > div.score_result > ul > li > div.score_reple > dl > dt > em')[1+(2*i)].text
      heart = soup.select('body > div > div > div.score_result > ul > li > div.btn_area > a._sympathyButton > strong')[i].text
      hate = soup.select('body > div > div > div.score_result > ul > li > div.btn_area > a._notSympathyButton > strong')[i].text

      movie_df = movie_df.append({'score' : score,'sentence' : sentence,'id' : id, 'date' : date, 'heart' : heart, 'hate' : hate}, ignore_index = True)
    
  print('crawling_time : ', time.time() - start_time)

  return movie_df

In [ ]:
naver_movie_review = crawling('https://movie.naver.com/movie/bi/mi/pointWriteFormList.naver?code=74977&amp;type=after&amp;isActualPointWriteExecute=false&amp;isMileageSubscriptionAlready=false&amp;isMileageSubscriptionReject=false')

In [57]:
naver_movie_review

,score,sentence,id,date,heart,hate
0,10,CG에 돈 많이 썼다는건 개구라임. 다만 직접 판도라가서 수중촬영하긴 빡쎘을거 같음...,archijun(arch****),2022.12.15 17:14,12445,920
1,10,말이안되네 ㅋㅋㅋ 어케만든겨,은덕(dmse****),2022.12.14 11:33,4270,251
2,10,제일궁금했던건 2009년도에도 대단했던 CG기술이 2022년도에 얼마나 대단해졌을까...,hwle****,2022.12.14 10:51,4050,349
3,10,미래에도 영화관이 있어야 되는 이유를 보여준 영화.,버츠비이이이(ydg3****),2022.12.14 13:23,3806,216
4,10,믿고보는 아바타 아쿠아리움에 있는줄..3시간이 어떻게 간건지 모르겠어요,nice****,2022.12.14 13:08,2853,205
...,...,...,...,...,...,...
21179,4,재미는 없는데 러닝티임은 길어서 결말 안보고 그냥 화장실 간김에 나왔어요.,Titanium(jmpa****),2022.12.18 19:49,37,85
21180,6,배신자 제이크 설리 및 파란 제노 박멸을 위해 몸 바쳐 뛰신 쿼리치 대령님의 투혼만...,Wukko(silv****),2022.12.14 12:36,15,67
21181,1,"리뷰에 속아 돈과 시간, 그리고 함께하는 사람을 잃지마세요 꼭 다른것을 보시길",wnsd****,2022.12.25 02:15,58,112
21182,2,끝까지 못보겠음.. 어디가 재밌다는 건지?,데비(real****),2022.12.15 18:27,57,137


In [58]:
naver_movie_review.to_csv('./네이버 영화 리뷰.csv',index=False)